In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

In [78]:
# 使用Selenium启动浏览器
url = "https://finance.yahoo.com/"
driver = webdriver.Chrome()  # 你需要安装Chrome浏览器驱动程序
driver.get(url)

# 初始化滚动高度
last_height = driver.execute_script("return document.body.scrollHeight")

# 等待加载
time.sleep(2)

# 模拟滚动并等待页面加载
while True:
    # 模拟滚动到页面底部
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # 等待加载
    time.sleep(2)

    # 获取新的页面高度
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 如果页面没有继续增加高度，说明已经加载完毕
    if new_height == last_height:
        break
    
    last_height = new_height

In [80]:
# 使用Beautiful Soup解析HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')
print(soup)

<html class="chrome desktop JsEnabled themelight layoutEnhance(TwoColumnLayout) CollapsibleUh onDemandFocusSupport HideNavrail hasScrolled scrollDown" data-color-theme="light" id="atomic" lang="en-US"><head prefix="og: https://ogp.me/ns#"><script async="" src="https://hb.yahoo.net/tcb.js?&amp;cb=window.advBidxc.multiBidTemplatefetch&amp;req=T32V66U_300x600&amp;v=118" type="text/javascript"></script><script>window.performance && window.performance.mark && window.performance.mark('PageStart');</script><meta charset="utf-8"/><title>Yahoo Finance - Stock Market Live, Quotes, Business &amp; Finance News</title><meta content="401k, Business, Financial Information, Investing, Investor, Market News, Stock Research, Stock Valuation, business news, economy, finance, investment tools, mortgage, mutual funds, personal finance, quote, real estate, retirement, stock, stocks, Suze Orman, tax, track portfolio" name="keywords"/><meta content="on" http-equiv="x-dns-prefetch-control"/><meta content="on" 

In [81]:
# 找到所有的文章链接
article_links = []
for link in soup.find_all("a"):
    href = link.get("href")
    if href and "/news/" in href:  # 筛选出文章链接
        # 使用urljoin函数建立完整的URL
        full_url = urljoin(url, href)
        article_links.append(full_url)
        
print(article_links)

['https://finance.yahoo.com/news/gold-is-approaching-all-time-highs-but-its-not-the-time-to-buy-strategist-says-192731810.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/inflation-is-higher-than-the-fed-wants-the-problem-could-be-its-target-185535285.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/dow-loses-nearly-300-points-after-bond-market-pummeling-stock-market-news-today-200534168.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/feds-mester-still-views-one-more-rate-hike-as-appropriate-161531253.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/1-white-house-asks-congress-145406375.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/us-futures-fall-as-week-of-battering-by-yields-nears-end-stock-market-news-today-111833294.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/jay-powell-will-keep-saying-inflation-is-too-high-until-consumers-believe-him-100020733.html?.tsrc=fin-notif', 'https://finance.yahoo.com/news/even-strong-earnings-might-not-be-abl

In [82]:
# 筛选包含关键字"war"的文章（不区分大小写）
keyword = "war"
filtered_articles = set()  # 使用集合来避免重复的文章

for article_link in article_links:
    article_response = requests.get(article_link)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')

    # 提取文章标题
    article_title = article_soup.find("h1", {"data-test-locator": "headline"})
    if article_title:
        article_title = article_title.text.strip().lower()  # 将标题转换为小写
    else:
        article_title = "No Title"

    try:
        # 提取文章内容
        article_content = article_soup.find("div", {"class": "caas-body"})
        if article_content:
            # 获取所有段落
            paragraphs = article_content.find_all("p")
            article_content_text = "\n".join([p.get_text() for p in paragraphs])  # 获取段落文本
            article_content_text = article_content_text.lower()  # 将内容转换为小写
        else:
            article_content_text = "No Content"
    except AttributeError:
        # 处理找不到文章内容的情况
        article_content_text = "No Content"

    if keyword in article_title or keyword in article_content_text:
        # 获取时间和作者
        article_time = article_soup.find("time")["datetime"]
        article_author = article_soup.find("span", {"class": "caas-author-byline-collapse"}).text.strip()

        filtered_articles.add((article_title, article_time, article_author, article_content_text))
        
print(filtered_articles)

{('us futures fall as week of battering by yields nears end: stock market news today', '2023-10-20T11:28:19.000Z', 'Karen Friar', 'us stocks slipped ahead of the bell on friday, with benchmark treasury yields on the retreat after spiking to 5% in the wake of comments by federal reserve chair jerome powell.\ndow jones industrial average (^dji) was down about 0.2%, while s&p 500 (^gspc) futures shed around 0.3%, both on track for a losing week after thursday\'s sell-off. contracts on the tech-heavy nasdaq 100 (^ndx) dropped almost 0.4%.\nstocks lost ground after powell signaled the fed is committed to its "higher for longer" rates stance, which spurred gains in treasury yields. the benchmark 10-year yield (^tnx) rose briefly to 5% late on thursday, a closely watched level not seen since july 2007.\n“the underlying message is \'don’t be looking for a bailout from the fed anytime soon,\'” greg whiteley, a portfolio manager at doubleline, told reuters. "that gives people the go ahead to tak

In [87]:
# 清除原有的CSV文件内容
with open("filtered_articles.csv", "w", newline='', encoding='utf-8') as csv_file:
    csv_file.truncate(0)  # 清空文件内容

# 清除原有的JSON文件内容
with open("filtered_articles.json", "w", encoding='utf-8') as json_file:
    json_file.truncate(0)  # 清空文件内容

In [88]:
# 储存筛选后的文章为CSV文件
with open("filtered_articles.csv", "w", newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Title", "Time", "Author", "Content"])
    for article in filtered_articles:
        writer.writerow(article)


In [89]:
# 储存筛选后的文章为JSON文件
filtered_articles_list = [{"Title": article[0], "Time": article[1], "Author": article[2], "Content": article[3]} for article in filtered_articles]
with open("filtered_articles.json", "w", encoding='utf-8') as json_file:
    json.dump(filtered_articles_list, json_file, ensure_ascii=False, indent=4)

print("筛选后的文章数据已成功储存为CSV和JSON文件。")

筛选后的文章数据已成功储存为CSV和JSON文件。


In [90]:
# 关闭浏览器
driver.quit()

# 打印抓取到的文章内容
for article in filtered_articles:
    print("Title:", article[0])
    print("Time:", article[1])
    print("Author:", article[2])
    print("Content:\n", article[3])
    print("=" * 50)

Title: us futures fall as week of battering by yields nears end: stock market news today
Time: 2023-10-20T11:28:19.000Z
Author: Karen Friar
Content:
 us stocks slipped ahead of the bell on friday, with benchmark treasury yields on the retreat after spiking to 5% in the wake of comments by federal reserve chair jerome powell.
dow jones industrial average (^dji) was down about 0.2%, while s&p 500 (^gspc) futures shed around 0.3%, both on track for a losing week after thursday's sell-off. contracts on the tech-heavy nasdaq 100 (^ndx) dropped almost 0.4%.
stocks lost ground after powell signaled the fed is committed to its "higher for longer" rates stance, which spurred gains in treasury yields. the benchmark 10-year yield (^tnx) rose briefly to 5% late on thursday, a closely watched level not seen since july 2007.
“the underlying message is 'don’t be looking for a bailout from the fed anytime soon,'” greg whiteley, a portfolio manager at doubleline, told reuters. "that gives people the go